In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from math import sqrt
from scipy.stats import pearsonr, spearmanr

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LassoLars, LinearRegression, TweedieRegressor

import acquire
import prepare
import explore
import model

%matplotlib inline
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings("ignore")

seed = 42

In [2]:
df = acquire.get_zillow()

In [3]:
df.head()

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,10804942,3.0,2.0,1175.0,327859.0,1953.0,4050.73,6037.0
1,14207926,3.0,2.0,1630.0,63948.0,1953.0,852.29,6037.0
2,10986950,2.0,2.0,1206.0,356218.0,1954.0,4393.33,6037.0
3,14207492,3.0,2.0,1790.0,242773.0,1964.0,3037.37,6037.0
4,11147606,3.0,2.0,1400.0,251246.0,1953.0,3081.31,6037.0


In [4]:
df = prepare.prep_zillow(df)

In [5]:
df.head()

,bedrooms,bathrooms,sqft,tax_value,year_built,fips
0,3.0,2.0,1175.0,327859.0,1953.0,6037.0
1,3.0,2.0,1630.0,63948.0,1953.0,6037.0
2,2.0,2.0,1206.0,356218.0,1954.0,6037.0
3,3.0,2.0,1790.0,242773.0,1964.0,6037.0
4,3.0,2.0,1400.0,251246.0,1953.0,6037.0


In [6]:
cols = ['bedrooms', 'bathrooms', 'sqft', 'tax_value', 'year_built', 'fips']

In [7]:
df = prepare.remove_outliers(df, 2, cols)

In [8]:
df

,bedrooms,bathrooms,sqft,tax_value,year_built,fips
0,3.0,2.0,1175.0,327859.0,1953.0,6037.0
1,3.0,2.0,1630.0,63948.0,1953.0,6037.0
2,2.0,2.0,1206.0,356218.0,1954.0,6037.0
3,3.0,2.0,1790.0,242773.0,1964.0,6037.0
4,3.0,2.0,1400.0,251246.0,1953.0,6037.0
...,...,...,...,...,...,...
56075,3.0,2.0,1400.0,318206.0,1951.0,6037.0
56076,4.0,2.0,1446.0,140804.0,1951.0,6037.0
56077,4.0,2.0,1584.0,412114.0,1955.0,6037.0
56078,4.0,2.0,1584.0,186627.0,1955.0,6037.0


In [9]:
train, validate, test = prepare.subset_df(df)

(27650, 6) (9217, 6) (9217, 6)


In [10]:
X_train, y_train, X_validate, y_validate, X_test, y_test = model.xy_subsets(train, validate, test, 'tax_value')

In [11]:
cols = ['sqft', 'year_built']

X_train_scaled, X_validate_scaled, X_test_scaled = prepare.scale_data(X_train, X_validate, X_test, MinMaxScaler(), cols)

In [12]:
X_train_scaled.shape, X_validate_scaled.shape, X_test_scaled.shape

((27650, 5), (9217, 5), (9217, 5))

In [17]:
X_train_scaled

,bedrooms,bathrooms,sqft,year_built,fips
39518,4.0,3.0,0.410496,0.603604,6059.0
36470,4.0,2.0,0.378717,0.819820,6037.0
14646,4.0,3.0,0.664431,0.765766,6037.0
6714,4.0,2.0,0.281633,0.378378,6037.0
44146,3.0,2.0,0.346064,0.324324,6059.0
...,...,...,...,...,...
14332,3.0,2.0,0.300000,0.468468,6037.0
54643,5.0,2.0,0.521283,0.450450,6059.0
46604,3.0,2.0,0.394169,0.423423,6037.0
927,3.0,1.0,0.362391,0.405405,6037.0


In [14]:
def rfe(n_features, X_train, y_train):
    
    lm = LinearRegression()
    rfe = RFE(lm, n_features_to_select=n_features)

    rfe.fit(X_train, y_train)

    ranks = rfe.ranking_
    columns = X_train.columns.tolist()
    
    feature_ranks = pd.DataFrame({'ranking': ranks,
                              'feature': columns})

    feature_ranks = feature_ranks.sort_values('ranking')

    return pd.DataFrame(feature_ranks).head(n_features)

In [15]:
def f_selector(k, X_train, y_train):
    
    f_selector = SelectKBest(f_regression, k=k)

    f_selector.fit(X_train_scaled, y_train)

    f_select_mask = f_selector.get_support()

    select_k_best_features = X_train_scaled.iloc[:,f_select_mask]
    
    #print(select_k_best_features.head(k))
    
    return pd.DataFrame(select_k_best_features)

In [128]:
f_selector(4, X_train_scaled, y_train)

,bedrooms,bathrooms,sqft,year_built
12634,2.0,2.0,-0.399371,0.148148
18194,3.0,2.0,0.077568,0.185185
43422,3.0,2.5,-0.028302,1.333333
41163,4.0,2.0,-0.266247,-0.148148
47308,3.0,2.0,0.001048,-0.296296
...,...,...,...,...
45030,5.0,6.0,3.242138,1.814815
54685,3.0,2.0,0.004193,0.407407
38399,4.0,4.0,1.351153,-0.222222
862,5.0,6.0,4.023061,1.629630


In [130]:
X_train_scaled = X_train_scaled[['bedrooms', 'sqft', 'bathrooms', 'year_built']]

In [19]:
X_train_scaled = pd.get_dummies(X_train_scaled, columns=['bedrooms', 'bathrooms'])

In [20]:
X_train_scaled

,sqft,year_built,fips,bedrooms_2.0,bedrooms_3.0,bedrooms_4.0,bedrooms_5.0,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5
39518,0.410496,0.603604,6059.0,0,0,1,0,0,0,0,0,1,0,0,0
36470,0.378717,0.819820,6037.0,0,0,1,0,0,0,1,0,0,0,0,0
14646,0.664431,0.765766,6037.0,0,0,1,0,0,0,0,0,1,0,0,0
6714,0.281633,0.378378,6037.0,0,0,1,0,0,0,1,0,0,0,0,0
44146,0.346064,0.324324,6059.0,0,1,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14332,0.300000,0.468468,6037.0,0,1,0,0,0,0,1,0,0,0,0,0
54643,0.521283,0.450450,6059.0,0,0,0,1,0,0,1,0,0,0,0,0
46604,0.394169,0.423423,6037.0,0,1,0,0,0,0,1,0,0,0,0,0
927,0.362391,0.405405,6037.0,0,1,0,0,1,0,0,0,0,0,0,0


In [22]:
def lin_regression(X_train, y_train):
    

    lm = LinearRegression()

    lm.fit(X_train, y_train)

    lm_preds = lm.predict(X_train)

    preds_df = pd.DataFrame({'actual': y_train,
                             'lm_preds': lm_preds})

    preds_df['baseline_avg'] = y_train.median()
    
    return preds_df

In [25]:
preds_df = lin_regression(X_train_scaled, y_train)

In [26]:
lm_rmse = sqrt(mean_squared_error(preds_df['lm_preds'], preds_df['actual']))

lm_rmse

200631.59415393448

In [224]:
sqrt(mean_squared_error(preds_df['actual'], preds_df['baseline_avg']))

555332.5503557529

In [229]:
preds_df['baseline_median'] = y_train.median()

In [228]:
preds_df

,actual,lm_preds,baseline_avg,lasso_preds,tweedie_norm_preds,tweedie_poisson_preds,poly_preds,lasso_poly,tweedie_norm_poly_preds,tweedie_poisson_poly_preds,baseline_median
12634,832186.0,3.109996e+05,453681.877755,2.944441e+05,420789.891095,453681.877755,3.109996e+05,2.946687e+05,420766.989135,453681.877755,351356.679349
18194,543910.0,3.874733e+05,453681.877755,3.960113e+05,421106.247986,453681.877755,3.874733e+05,3.960526e+05,421126.479023,453681.877755,351356.679349
43422,559167.0,3.604702e+05,453681.877755,3.723558e+05,466545.538714,453681.877755,3.604702e+05,3.727009e+05,466554.596275,453681.877755,351356.679349
41163,131876.0,2.274146e+05,453681.877755,2.585272e+05,448834.071685,453681.877755,2.274146e+05,2.586159e+05,448819.437258,453681.877755,351356.679349
47308,583042.0,3.810907e+05,453681.877755,3.731611e+05,413508.505594,453681.877755,3.810907e+05,3.728441e+05,413527.326264,453681.877755,351356.679349
...,...,...,...,...,...,...,...,...,...,...,...
45030,4275000.0,1.991996e+06,453681.877755,1.807953e+06,520294.566254,453681.877755,1.991996e+06,1.807698e+06,520181.152622,453681.877755,351356.679349
54685,234378.0,3.601889e+05,453681.877755,3.720615e+05,421678.103620,453681.877755,3.601889e+05,3.722948e+05,421697.981084,453681.877755,351356.679349
38399,1180635.0,9.093724e+05,453681.877755,7.921788e+05,508336.165238,453681.877755,9.093724e+05,7.923620e+05,508289.363325,453681.877755,351356.679349
862,1098926.0,2.230352e+06,453681.877755,2.060027e+06,520141.299236,453681.877755,2.230352e+06,2.059504e+06,520026.719092,453681.877755,351356.679349


In [ ]:
sqrt(mean_squared_error(preds_df['actual'], preds_df['baseline_median']))

In [170]:
lm_rmse = sqrt(mean_squared_error(preds_df['lm_preds'], preds_df['actual']))

lm_rmse

399320.6681446739

In [171]:
preds_df

,actual,lm_preds,baseline_avg
12634,832186.0,3.109996e+05,453681.877755
18194,543910.0,3.874733e+05,453681.877755
43422,559167.0,3.604702e+05,453681.877755
41163,131876.0,2.274146e+05,453681.877755
47308,583042.0,3.810907e+05,453681.877755
...,...,...,...
45030,4275000.0,1.991996e+06,453681.877755
54685,234378.0,3.601889e+05,453681.877755
38399,1180635.0,9.093724e+05,453681.877755
862,1098926.0,2.230352e+06,453681.877755


In [172]:
lasso = LassoLars(alpha=.33)

lasso.fit(X_train_scaled1, y_train)

lasso_preds = lasso.predict(X_train_scaled1)

preds_df['lasso_preds'] = lasso_preds

preds_df.head()

,actual,lm_preds,baseline_avg,lasso_preds
12634,832186.0,310999.610612,453681.877755,294444.149871
18194,543910.0,387473.273456,453681.877755,396011.325054
43422,559167.0,360470.231972,453681.877755,372355.816999
41163,131876.0,227414.595718,453681.877755,258527.216446
47308,583042.0,381090.740151,453681.877755,373161.130022


In [173]:
lasso_rmse = sqrt(mean_squared_error(preds_df['actual'], preds_df['lasso_preds']))
lasso_rmse

405831.84183986206

In [174]:
tweedie = TweedieRegressor(power=0)

tweedie.fit(X_train_scaled1, y_train)

tweedie_preds = tweedie.predict(X_train_scaled1)

preds_df['tweedie_norm_preds'] = tweedie_preds

preds_df.head()

,actual,lm_preds,baseline_avg,lasso_preds,tweedie_norm_preds
12634,832186.0,310999.610612,453681.877755,294444.149871,420789.891095
18194,543910.0,387473.273456,453681.877755,396011.325054,421106.247986
43422,559167.0,360470.231972,453681.877755,372355.816999,466545.538714
41163,131876.0,227414.595718,453681.877755,258527.216446,448834.071685
47308,583042.0,381090.740151,453681.877755,373161.130022,413508.505594


In [175]:
tweedie_norm = sqrt(mean_squared_error(preds_df['actual'], preds_df['tweedie_norm_preds']))

tweedie_norm

543356.7327284052

In [176]:
tweedie = TweedieRegressor(power=1)

tweedie.fit(X_train_scaled1, y_train)

tweedie_preds = tweedie.predict(X_train_scaled1)

preds_df['tweedie_poisson_preds'] = tweedie_preds

preds_df.head()

,actual,lm_preds,baseline_avg,lasso_preds,tweedie_norm_preds,tweedie_poisson_preds
12634,832186.0,310999.610612,453681.877755,294444.149871,420789.891095,453681.877755
18194,543910.0,387473.273456,453681.877755,396011.325054,421106.247986,453681.877755
43422,559167.0,360470.231972,453681.877755,372355.816999,466545.538714,453681.877755
41163,131876.0,227414.595718,453681.877755,258527.216446,448834.071685,453681.877755
47308,583042.0,381090.740151,453681.877755,373161.130022,413508.505594,453681.877755


In [177]:
tweedie_poisson = sqrt(mean_squared_error(preds_df['actual'], preds_df['tweedie_poisson_preds']))

tweedie_poisson

555332.5503557529

In [197]:
pf = PolynomialFeatures(degree=1)

pf.fit(X_train_scaled1, y_train)

X_polynomial = pf.transform(X_train_scaled1)

In [198]:
X_polynomial.shape

(33441, 40)

In [199]:
lm2 = LinearRegression()

lm2.fit(X_polynomial, y_train)

preds_df['poly_preds'] = lm2.predict(X_polynomial)

preds_df.head()

,actual,lm_preds,baseline_avg,lasso_preds,tweedie_norm_preds,tweedie_poisson_preds,poly_preds,lasso_poly,tweedie_norm_poly_preds,tweedie_poisson_poly_preds
12634,832186.0,310999.610612,453681.877755,294444.149871,420789.891095,453681.877755,310999.610612,295944.078137,317899.452214,453681.877755
18194,543910.0,387473.273456,453681.877755,396011.325054,421106.247986,453681.877755,387473.273456,379633.493136,330283.994582,453681.877755
43422,559167.0,360470.231972,453681.877755,372355.816999,466545.538714,453681.877755,360470.231972,374745.020015,473083.596477,453681.877755
41163,131876.0,227414.595718,453681.877755,258527.216446,448834.071685,453681.877755,227414.595718,323724.882302,268233.287765,453681.877755
47308,583042.0,381090.740151,453681.877755,373161.130022,413508.505594,453681.877755,381090.740151,361611.324417,303145.983877,453681.877755


In [200]:
lm2_rmse = sqrt(mean_squared_error(preds_df['actual'], preds_df['poly_preds']))

lm2_rmse

399320.6681446739

In [201]:
lasso2 = LassoLars(alpha=.33)

lasso2.fit(X_polynomial, y_train)

lasso2_preds = lasso2.predict(X_polynomial)

preds_df['lasso_poly'] = lasso2_preds

preds_df.head()

,actual,lm_preds,baseline_avg,lasso_preds,tweedie_norm_preds,tweedie_poisson_preds,poly_preds,lasso_poly,tweedie_norm_poly_preds,tweedie_poisson_poly_preds
12634,832186.0,310999.610612,453681.877755,294444.149871,420789.891095,453681.877755,310999.610612,294668.662878,317899.452214,453681.877755
18194,543910.0,387473.273456,453681.877755,396011.325054,421106.247986,453681.877755,387473.273456,396052.565312,330283.994582,453681.877755
43422,559167.0,360470.231972,453681.877755,372355.816999,466545.538714,453681.877755,360470.231972,372700.914603,473083.596477,453681.877755
41163,131876.0,227414.595718,453681.877755,258527.216446,448834.071685,453681.877755,227414.595718,258615.936908,268233.287765,453681.877755
47308,583042.0,381090.740151,453681.877755,373161.130022,413508.505594,453681.877755,381090.740151,372844.053361,303145.983877,453681.877755


In [202]:
lasso2_rmse = sqrt(mean_squared_error(preds_df['actual'], preds_df['lasso_poly']))

lasso2_rmse

405915.3165462692

In [203]:
tweedie = TweedieRegressor(power=0)

tweedie.fit(X_polynomial, y_train)

tweedie_preds = tweedie.predict(X_polynomial)

preds_df['tweedie_norm_poly_preds'] = tweedie_preds

preds_df.head()

,actual,lm_preds,baseline_avg,lasso_preds,tweedie_norm_preds,tweedie_poisson_preds,poly_preds,lasso_poly,tweedie_norm_poly_preds,tweedie_poisson_poly_preds
12634,832186.0,310999.610612,453681.877755,294444.149871,420789.891095,453681.877755,310999.610612,294668.662878,420766.989135,453681.877755
18194,543910.0,387473.273456,453681.877755,396011.325054,421106.247986,453681.877755,387473.273456,396052.565312,421126.479023,453681.877755
43422,559167.0,360470.231972,453681.877755,372355.816999,466545.538714,453681.877755,360470.231972,372700.914603,466554.596275,453681.877755
41163,131876.0,227414.595718,453681.877755,258527.216446,448834.071685,453681.877755,227414.595718,258615.936908,448819.437258,453681.877755
47308,583042.0,381090.740151,453681.877755,373161.130022,413508.505594,453681.877755,381090.740151,372844.053361,413527.326264,453681.877755


In [204]:
tweedie_norm_poly = sqrt(mean_squared_error(preds_df['actual'], preds_df['tweedie_norm_poly_preds']))

tweedie_norm_poly

543368.1477620819

In [205]:
tweedie = TweedieRegressor(power=1)

tweedie.fit(X_polynomial, y_train)

tweedie_preds = tweedie.predict(X_polynomial)

preds_df['tweedie_poisson_poly_preds'] = tweedie_preds

preds_df.head()

,actual,lm_preds,baseline_avg,lasso_preds,tweedie_norm_preds,tweedie_poisson_preds,poly_preds,lasso_poly,tweedie_norm_poly_preds,tweedie_poisson_poly_preds
12634,832186.0,310999.610612,453681.877755,294444.149871,420789.891095,453681.877755,310999.610612,294668.662878,420766.989135,453681.877755
18194,543910.0,387473.273456,453681.877755,396011.325054,421106.247986,453681.877755,387473.273456,396052.565312,421126.479023,453681.877755
43422,559167.0,360470.231972,453681.877755,372355.816999,466545.538714,453681.877755,360470.231972,372700.914603,466554.596275,453681.877755
41163,131876.0,227414.595718,453681.877755,258527.216446,448834.071685,453681.877755,227414.595718,258615.936908,448819.437258,453681.877755
47308,583042.0,381090.740151,453681.877755,373161.130022,413508.505594,453681.877755,381090.740151,372844.053361,413527.326264,453681.877755


In [206]:
tweedie_poisson_poly = sqrt(mean_squared_error(preds_df['actual'], preds_df['tweedie_poisson_poly_preds']))

tweedie_poisson_poly

555332.5503557529

In [156]:
results = pd.DataFrame({'model':['linear', 'lasso', 'linear_poly', 'lasso_poly', 'tweedie_norm', 'tweedie_poisson'],
              'rmse':[lm_rmse, lasso_rmse, lm2_rmse, lasso2_rmse, tweedie_norm, tweedie_poisson]})

results

,model,rmse
0,linear,399344.944565
1,lasso,400633.520647
2,linear_poly,365184.743116
3,lasso_poly,392023.879116
4,tweedie_norm,457441.218454
5,tweedie_poisson,421334.822389


In [188]:
results = pd.DataFrame({'model':['linear', 'lasso', 'linear_poly', 'lasso_poly', 'tweedie_norm', 'tweedie_poisson'],
              'rmse':[lm_rmse, lasso_rmse, lm2_rmse, lasso2_rmse, tweedie_norm, tweedie_poisson]})

results

,model,rmse
0,linear,399320.668145
1,lasso,405831.841840
2,linear_poly,361905.218844
3,lasso_poly,386954.493355
4,tweedie_norm,543356.732728
5,tweedie_poisson,555332.550356


In [150]:
results = pd.DataFrame({'model':['linear', 'lasso', 'linear_poly', 'lasso_poly', 'tweedie_norm', 'tweedie_poisson'],
              'rmse':[lm_rmse, lasso_rmse, lm2_rmse, lasso2_rmse, tweedie_norm, tweedie_poisson]})

results

,model,rmse
0,linear,399344.944565
1,lasso,400633.520647
2,linear_poly,376886.644089
3,lasso_poly,392802.327333
4,tweedie_norm,457441.218454
5,tweedie_poisson,421334.822389
